# Considering Bias in Data

The following analysis will help us gain a better understanding of how biases can manifest in datasets, particularly in the context of publicly contributed information sources such as Wikipedia. 

The analysis will cover the following aspects:
1. Coverage Disparity of US Cities on Wikipedia: An assessment of the differences in coverage among various US states in relation to their population.
2. Quality Disparity of Wikipedia Articles: Examination of the variation in the proportion of high-quality articles about cities across different states.
3. Ranking of US Geographic Regions: A ranking system based on articles-per-person ratio and the proportion of high-quality articles to offer insights into the bias and disparities present in Wikipedia data.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('wp_scored_city_articles_by_state.csv')
df.head()

,state,population,regional_division,article_title,revision_id,article_quality
0,Alabama,5074296,East South Central,"Abbeville, Alabama",1171163550,C
1,Alabama,5074296,East South Central,"Adamsville, Alabama",1177621427,C
2,Alabama,5074296,East South Central,"Addison, Alabama",1168359898,C
3,Alabama,5074296,East South Central,"Akron, Alabama",1165909508,GA
4,Alabama,5074296,East South Central,"Alabaster, Alabama",1179139816,C


## Top 10 US states by coverage
The 10 US states with the highest total articles per capita (in descending order)

In [3]:
# Group by state and count the number of articles
articles_per_state = df.groupby('state')['article_title'].count().reset_index()

# Getting unique values of state along with corresponding population
population_per_state= df[['state', 'population']].drop_duplicates()

# Merge data
articles_per_capita = pd.merge(articles_per_state,population_per_state, on='state')

# Calculate the article per capita
articles_per_capita['article_per_capita'] = articles_per_capita['article_title'] / articles_per_capita['population']

articles_per_capita.head()


,state,article_title,population,article_per_capita
0,Alabama,461,5074296,0.000091
1,Alaska,149,733583,0.000203
2,Arizona,91,7359197,0.000012
3,Arkansas,500,3045637,0.000164
4,California,482,39029342,0.000012


In [9]:
top_10=articles_per_capita.sort_values(by='article_per_capita', ascending=False)[:10].reset_index(drop=True)
top_10

,state,article_title,population,article_per_capita
0,Vermont,329,647064,0.000508
1,North Dakota,356,779261,0.000457
2,Maine,483,1385340,0.000349
3,South Dakota,311,909824,0.000342
4,Iowa,1043,3200517,0.000326
5,Alaska,149,733583,0.000203
6,Pennsylvania,2556,12972008,0.000197
7,Michigan,1773,10034113,0.000177
8,Wyoming,99,581381,0.000170
9,New Hampshire,234,1395231,0.000168


## Bottom 10 US states by coverage
The 10 US states with the lowest total articles per capita (in ascending order)

In [10]:
bottom_10=articles_per_capita.sort_values(by='article_per_capita', ascending=True)[:10].reset_index(drop=True)
bottom_10

,state,article_title,population,article_per_capita
0,North Carolina,50,10698973,0.000005
1,Nevada,19,3177772,0.000006
2,California,482,39029342,0.000012
3,Arizona,91,7359197,0.000012
4,Virginia,133,8683619,0.000015
5,Florida,412,22244823,0.000019
6,Oklahoma,75,4019800,0.000019
7,Kansas,63,2937150,0.000021
8,Maryland,157,6164660,0.000025
9,Wisconsin,192,5892539,0.000033


## Top 10 US states by high quality
The 10 US states with the highest high quality articles per capita (in descending order). 

For this analysis you "high quality" articles are articles that ORES predicted in either the "FA" (featured article) or "GA" (good article) classes.



In [12]:
# Group by state and count the number of high-quality articles
high_quality_articles = df[df['article_quality'].isin(['GA', 'FA'])].groupby('state')['article_quality'].count().reset_index()

# Merge with population data
hq_articles_per_capita = pd.merge(high_quality_articles,population_per_state, on='state')

# Calculate the high quality article per capita
hq_articles_per_capita['article_quality'] = hq_articles_per_capita['article_quality'] / hq_articles_per_capita['population']

hq_articles_per_capita.head()

,state,article_quality,population
0,Alabama,0.000010,5074296
1,Alaska,0.000042,733583
2,Arizona,0.000003,7359197
3,Arkansas,0.000024,3045637
4,California,0.000004,39029342


In [15]:
top_10_hq=hq_articles_per_capita.sort_values(by='article_quality', ascending=False)[:10].reset_index(drop=True)
top_10_hq

,state,article_quality,population
0,Vermont,0.000070,647064
1,Wyoming,0.000067,581381
2,South Dakota,0.000062,909824
3,West Virginia,0.000059,1775156
4,Montana,0.000049,1122867
5,New Hampshire,0.000045,1395231
6,Pennsylvania,0.000044,12972008
7,Missouri,0.000043,6177957
8,Alaska,0.000042,733583
9,New Jersey,0.000041,9261699


## Bottom 10 US states by high quality
The 10 US states with the lowest high quality articles per capita (in ascending order)


In [16]:
bottom_10_hq=hq_articles_per_capita.sort_values(by='article_quality', ascending=True)[:10].reset_index(drop=True)
bottom_10_hq

,state,article_quality,population
0,North Carolina,0.000002,10698973
1,Virginia,0.000002,8683619
2,Nevada,0.000003,3177772
3,Arizona,0.000003,7359197
4,California,0.000004,39029342
5,Florida,0.000005,22244823
6,New York,0.000006,19677151
7,Maryland,0.000007,6164660
8,Kansas,0.000007,2937150
9,Oklahoma,0.000008,4019800


## Census divisions by total coverage
A rank ordered list of US census divisions (in descending order) by total articles per capita

In [4]:
# Group by region and count the number of articles
articles_per_region = df.groupby('regional_division')['article_title'].count().reset_index()

## Group by region and sum the population
population_per_region= df.groupby('regional_division')['population'].sum().reset_index()

# Merge data
articles_per_region = pd.merge(articles_per_region,population_per_region, on='regional_division')

# Calculate the article per capita
articles_per_region['article_per_capita'] = articles_per_region['article_title'] / articles_per_region['population']

# Convert scientific notation to decimal
articles_per_region['article_per_capita'] = articles_per_region['article_per_capita'].apply(lambda x: '%.9f' % x)

articles_per_region

,regional_division,article_title,population,article_per_capita
0,East North Central,4754,50000102986,0.000000095
1,East South Central,1529,7567764699,0.000000202
2,Middle Atlantic,3781,51386647495,0.000000074
3,Mountain,1189,4100790927,0.000000290
4,New England,1437,3708797804,0.000000387
5,Pacific,1304,22348595190,0.000000058
6,South Atlantic,1850,19595194539,0.000000094
7,West North Central,3578,14841264104,0.000000241
8,West South Central,2103,39975932892,0.000000053


## Census divisions by high quality coverage
Rank ordered list of US census divisions (in descending order) by high quality articles per capita

In [5]:
# Group by state and count the number of high-quality articles
hq_articles_region = df[df['article_quality'].isin(['GA', 'FA'])].groupby('regional_division')['article_title'].count().reset_index()

# Merge with population data
hq_articles_region = pd.merge(hq_articles_region,population_per_region, on='regional_division')

# Calculate the high quality article per capita
hq_articles_region['hq_article_per_capita'] = hq_articles_region['article_title'] / hq_articles_region['population']

# Convert scientific notation to decimal
hq_articles_region['hq_article_per_capita'] = hq_articles_region['hq_article_per_capita'].apply(lambda x: '%.9f' % x)


hq_articles_region

,regional_division,article_title,population,hq_article_per_capita
0,East North Central,711,50000102986,0.000000014
1,East South Central,315,7567764699,0.000000042
2,Middle Atlantic,1056,51386647495,0.000000021
3,Mountain,336,4100790927,0.000000082
4,New England,225,3708797804,0.000000061
5,Pacific,491,22348595190,0.000000022
6,South Atlantic,524,19595194539,0.000000027
7,West North Central,638,14841264104,0.000000043
8,West South Central,638,39975932892,0.000000016
